In [6]:
from bci_aic3.config import load_processing_config
from bci_aic3.paths import LABEL_MAPPING_PATH, MI_CONFIG_PATH
from bci_aic3.util import read_json_to_dict

from bci_aic3.data import load_raw_data
from bci_aic3.paths import RAW_DATA_DIR

In [7]:

processing_config = load_processing_config(MI_CONFIG_PATH)
label_mapping = read_json_to_dict(LABEL_MAPPING_PATH)

In [8]:
train, val, _ = load_raw_data(
    base_path=RAW_DATA_DIR,
    task_type="MI",
    label_mapping=label_mapping,
)


100%|██████████| 50/50 [00:02<00:00, 18.85it/s]


In [9]:
import numpy as np
import mne
from scipy.signal import butter, filtfilt
from sklearn.base import BaseEstimator, TransformerMixin
from mne.preprocessing import ICA
from mne.decoding import CSP

from torch.utils.data import DataLoader

from bci_aic3.config import ProcessingConfig
from bci_aic3.data import BCIDataset
from bci_aic3.paths import PROCESSED_DATA_DIR

In [47]:
class MIBCIPreprocessor(BaseEstimator, TransformerMixin):
    """
    A preprocessing pipeline for Motor Imagery BCI.

    This pipeline applies the following steps:
    1. Converts NumPy array to MNE EpochsArray.
    2. Applies a notch filter to remove powerline noise.
    3. Applies a custom Butterworth bandpass filter.
    4. Fits and applies ICA to remove biological artifacts.
    5. Crops the epochs to the time window of interest.
    6. Fits and applies Common Spatial Patterns (CSP) for feature extraction.

    The output is ready for a classification model like EEGNet.
    """

    def __init__(self, config: ProcessingConfig):
        self.config = config
        self.ica = None
        self.csp = None
        self.info = None

    def _create_mne_epochs(self, X: np.ndarray) -> mne.EpochsArray:
        """Creates an MNE EpochsArray object from a NumPy array."""
        if self.info is None:
            self.info = mne.create_info(
                ch_names=self.config.ch_names, sfreq=self.config.sfreq, ch_types="eeg"
            )
            montage = mne.channels.make_standard_montage("standard_1020")
            self.info.set_montage(montage, on_missing="ignore")

        tmin_original = 0
        return mne.EpochsArray(X, self.info, tmin=tmin_original, verbose=False)

    def _apply_filters(self, epochs: mne.EpochsArray) -> mne.EpochsArray:
        """Applies notch and bandpass filters to the data array."""
        # Get data as a NumPy array to apply filters
        data = epochs.get_data(copy=True)

        # 1. Notch Filter using the top-level MNE function
        data_notched = mne.filter.notch_filter(
            data,
            Fs=self.config.sfreq,
            freqs=self.config.notch_freq,
            method="iir",  # IIR is generally faster for notch
            verbose=False,
        )

        # 2. Bandpass Filter (using a zero-phase Butterworth filter)
        b, a = butter(  # type: ignore
            self.config.filter_order,
            [self.config.bandpass_low, self.config.bandpass_high],
            btype="bandpass",
            fs=self.config.sfreq,
        )

        # Apply the filter forward and backward to each channel and epoch
        data_bandpassed = filtfilt(b, a, data_notched, axis=-1)

        # Create a new EpochsArray with the filtered data
        epochs_filtered = mne.EpochsArray(
            data_bandpassed, epochs.info, tmin=epochs.tmin, verbose=False
        )
        return epochs_filtered

    def fit(self, X: np.ndarray, y: np.ndarray):
        """
        Fits the ICA and CSP transformers on the training data.

        Args:
            X: EEG data, shape (n_epochs, n_channels, n_times)
            y: Labels, shape (n_epochs,)
        """
        # --- Step 1: Create MNE object and apply basic filters ---
        epochs = self._create_mne_epochs(X)
        epochs_filtered = self._apply_filters(epochs)

        # --- Step 2: Fit ICA for artifact removal ---
        self.ica = ICA(
            n_components=self.config.ica_n_components,
            random_state=self.config.ica_random_state,
            max_iter="auto",
        )
        self.ica.fit(epochs_filtered)

        # --- Step 3: Apply ICA and Crop Epochs ---
        epochs_ica = self.ica.apply(epochs_filtered.copy(), verbose=False)
        epochs_cropped = epochs_ica.copy().crop(
            tmin=self.config.tmin, 
            tmax=self.config.tmax,
            include_tmax=False,
        )

        # --- Step 4: Fit CSP ---
        self.csp = CSP(
            n_components=self.config.n_csp_components,
            reg=None,
            log=None, 
            norm_trace=False,
            transform_into='csp_space',
        )
        self.csp.fit(epochs_cropped.get_data(), y)

        return self

    def transform(self, X: np.ndarray) -> np.ndarray:
        """
        Applies the learned transformations (ICA, CSP) to the data.

        Args:
            X: EEG data, shape (n_epochs, n_channels, n_times)

        Returns:
            Spatially filtered time-series data, shape (n_epochs, n_csp_components, n_cropped_times)
        """
        if self.ica is None or self.csp is None:
            raise RuntimeError(
                "The preprocessor has not been fitted yet. Call fit() first."
            )

        # --- Step 1 & 2: Create MNE object and apply filters ---
        epochs = self._create_mne_epochs(X)
        epochs_filtered = self._apply_filters(epochs)

        # --- Step 3: Apply fitted ICA ---
        epochs_ica = self.ica.apply(epochs_filtered, verbose=False)

        # --- Step 4: Crop to time window of interest ---
        epochs_cropped = epochs_ica.crop(tmin=self.config.tmin,
                                         tmax=self.config.tmax, 
                                         include_tmax=False)

        # --- Step 5: Apply fitted CSP to get spatially filtered time series ---
        eegnet_input = self.csp.transform(epochs_cropped.get_data())

        return eegnet_input.astype(np.float32)

    def fit_transform(self, X, y=None, **fit_params):
        """Fit to data, then transform it."""
        return self.fit(X, y).transform(X)  # type: ignore

In [48]:

def preprocessing_pipeline(
    train_dataset: BCIDataset,
    validation_dataset: BCIDataset,
    task_type: str,
    processing_config: ProcessingConfig,
):
    train_loader = DataLoader(
        train_dataset, batch_size=len(train_dataset), shuffle=False
    )
    train_data, train_labels = next(iter(train_loader))

    val_loader = DataLoader(
        validation_dataset, batch_size=len(validation_dataset), shuffle=False
    )
    val_data, val_labels = next(iter(val_loader))

    train_data = train_data.numpy()
    train_labels = train_labels.numpy()

    val_data = val_data.numpy()
    val_labels = val_labels.numpy()

    preprocessor = MIBCIPreprocessor(processing_config)

    train_data_processed = preprocessor.fit_transform(train_data, train_labels)

    val_data_processed = preprocessor.transform(val_data)

    # Define the directory where the files will be saved
    output_dir = PROCESSED_DATA_DIR / task_type.upper()

    # Create the directory if it doesn't exist
    output_dir.mkdir(parents=True, exist_ok=True)

    # Save processed training data and labels
    processed_train_data_path = output_dir / "train_data.npy"
    processed_labels_path = output_dir / "validation_labels.npy"

    np.save(processed_train_data_path, train_data_processed)
    print(f"Processed data successfully saved at: {processed_train_data_path}")
    np.save(processed_labels_path, train_labels)
    print(f"Processed labels successfully saved at: {processed_labels_path}")

    # Save processed validation data and labels
    processed_val_data_path = output_dir / "validation_data.npy"
    processed_labels_path = output_dir / "validation_labels.npy"

    np.save(processed_val_data_path, val_data_processed)
    print(f"Processed data successfully saved at: {processed_val_data_path}")

    np.save(processed_labels_path, val_labels)
    print(f"Processed labels successfully saved at: {processed_labels_path}")


In [49]:
processing_config = load_processing_config(MI_CONFIG_PATH)
label_mapping = read_json_to_dict(LABEL_MAPPING_PATH)
task_type = "MI"


train_loader = DataLoader(
    train, batch_size=len(train), shuffle=False,
)
train_data, train_labels = next(iter(train_loader))

val_loader = DataLoader(
    val, batch_size=len(val), shuffle=False,
)
val_data, val_labels = next(iter(val_loader))


In [50]:

train_data = train_data.numpy()
train_labels = train_labels.numpy()

val_data = val_data.numpy()
val_labels = val_labels.numpy()

In [51]:
train_data.shape, train_labels.shape

((2400, 8, 2250), (2400,))

In [52]:
val_data.shape, val_labels.shape

((50, 8, 2250), (50,))

In [53]:
preprocessor = MIBCIPreprocessor(processing_config)

train_data_processed = preprocessor.fit_transform(train_data, train_labels)


Fitting ICA to data using 8 channels (please be patient, this may take a while)


C:\Users\mbadr\AppData\Local\Temp\ipykernel_22452\2849712291.py:83: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  self.ica.fit(epochs_filtered)


Selecting by number: 8 components
Fitting ICA took 79.3s.
Computing rank from data with rank=None
    Using tolerance 8.5e+02 (2.2e-16 eps * 8 dim * 4.8e+17  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.


In [54]:
train_data_processed.shape

(2400, 6, 750)

In [55]:
val_data_processed = preprocessor.transform(val_data)

In [56]:
val_data_processed.shape

(50, 6, 750)